# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
print(azureml.core.VERSION)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive import BanditPolicy
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-heart-failure'
project_folder = './mlops-capstone-project-folder'

experiment=Experiment(ws, experiment_name)
experiment

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-cpu"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)

In [ ]:
from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name = 'scikitlearn_env', file_path = './envFile.yml')
sklearn_env

In [ ]:
from azureml.core import ScriptRunConfig
src = ScriptRunConfig(source_directory='.',
                      compute_target=compute_target,
                      script='train.py',
                      arguments=['--C', 1.0, '--max_iter', 100],
                      environment=sklearn_env)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000), "--max_iter": choice(40,80,120,130,200)})

#TODO: Create your estimator and hyperdrive config
#estimator = <your estimator here>

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                         primary_metric_name="Accuracy",
                                         policy=early_termination_policy,
                                         max_total_runs=10,
                                         max_concurrent_runs=4,
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         run_config=src)

In [ ]:
#TODO: Submit your experiment
hyperDrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperDrive_run).show()
hyperDrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

In [ ]:
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength ("--C"):',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations:',best_run_metrics['Max iterations:'])
#joblib.dump(value=best_run.id, filename="./outputs/best_capstone_project_hyperdrive_model.joblib")

In [ ]:
the best model
print(os.getcwd())
os.makedirs('./hyperdrive-capstone-model',exist_ok=True)
best_run.download_file('/outputs/best_hyperdrive_model.joblib',os.path.join('./hyperdrive-capstone-model','best_hyperdrive_model.joblib'))

In [ ]:
#Register Model
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

bestModel = best_run.register_model(
                        model_path='./outputs/best_hyperdrive_model.joblib', 
                        model_name='HyperDrive_Best_Model',
                        tags={'Capstone HyperDrive AML Training tag':'Capstone Hyperdrive Auto ML'},
                        model_framework=Model.Framework.SCIKITLEARN,
                        description='heart faliure predication.',
                        properties={'Accuracy': best_run_metrics['Accuracy']}
                        )

print(bestModel)
print('Version:', bestModel.version)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

